In [1]:
import data_generation

q,p,y = data_generation.generate_data_RNN(seq_length=8,num_seq=131072)

10000 sequences generated
20000 sequences generated
30000 sequences generated
40000 sequences generated
50000 sequences generated
60000 sequences generated
70000 sequences generated
80000 sequences generated
90000 sequences generated
100000 sequences generated
110000 sequences generated
120000 sequences generated
130000 sequences generated


In [6]:
%load_ext autoreload
%autoreload 2

import h5py 

f = h5py.File('data/rnn_large_data.hdf5','w')
f['q'] = q
f['p'] = p
f['y'] = y

f.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import sys

sys.stdout.write(".........")
sys.stdout.flush()


...........

In [8]:
%load_ext autoreload
%autoreload 2

from pong_recurrent_convolution_net import *

model = PongRCNN(batch_size=64)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


pong_recurrent_convolution_net.py:48: UserWarning: pool_2d() will have the parameter ignore_border default value changed to True (currently False). To have consistent behavior with all Theano version, explicitly add the parameter ignore_border=True. On the GPU, using ignore_border=False is needed to use CuDNN. When using ignore_border=False and not using CuDNN, the only GPU combination supported is when `ds == st and padding == (0, 0) and mode == 'max'`. Otherwise, the convolution will be executed on CPU.
  self.POOL = T.signal.pool.pool_2d(self.LSTM_RCN3.output,(2,2))


In [15]:
q_0 = q[:,0:128]
p_0 = p[:,0:128]
y_0 = y[:,0:128]

print q_0.shape,p_0.shape,y_0.shape

loss = model.train(q_0,p_0,y_0,1e-1)

(8, 128, 32, 32) (8, 128, 2) (8, 128, 32, 32)


In [ ]:
alpha = 1e-1
lr_decay=0.99
num_epochs = 1

history = model.train_multibatch(q,p,y,alpha,lr_decay,num_epochs)

In [9]:
import numpy as np
from six.moves import cPickle
import subprocess

alpha = 1e-1
lr_decay=0.99
num_epochs = 1

i=1

while True: 
    i+1
    h = model.train_multibatch(q,p,y,alpha,1.0,1)
    history = np.concatenate((history, h),axis=0)
    alpha = alpha * lr_decay
    
    with open('history/6layer-RCN-'+str(i)+'.save','wb') as f: 
        cPickle.dump(history,f)
        
    model.save_weights('weights/6layer-RCN.hdf5')
    
    subprocess.call("aws s3 sync /home/ubuntu/pong_rnn/weights s3://model-checkpoints",shell=True)
    subprocess.call("aws s3 sync /home/ubuntu/pong_rnn/history s3://model-history",shell=True)

    
    

        
    

ZeroDivisionError: integer division or modulo by zero